# HomeWord 1 : Linear Regression
本次实验的目标： 由前9个小时的18个特征预测第10个小时的PM2.5
[参考](https://colab.research.google.com/drive/131sSqmrmWXfjFZ3jWSELl8cm0Ox5ah3C#scrollTo=NzvXP5Jya64j)

In [47]:
import sys
import pandas as pd
import numpy as np
from  tqdm import tqdm

## Preprocssing 数据预处理
取需要的数值部分，将'RAINFALL'全部设置为0

In [48]:
data = pd.read_csv('./train.csv', encoding='big5')

In [49]:
data.head()

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [50]:
# 抛去数据的前三列不要了
data = data.iloc[:, 3:]
# 将为NR的数据变为 0
data[data == 'NR'] = 0
raw_data = data.to_numpy()

## Extract Features 特征提取 - 1 
将原始的4320*18的资料依照每个月组成12个月 18*480的数据类型(12*18*480) 其中18是feature 480 是 20*24 得来的

In [51]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480]) # 创建一个空样本大小为18*480
    for day in range(20): # 一个月只取了20天
        # 从raw_data中取数据 将数据填充进 sample
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

## Extract Features 特征提取 - 2
每个月有480小时，每9个小时为一个data，每个月会有471个data，故总资料数为471\*12笔，而每笔数据有9\*18的feature(一个小时18个feature)

对应的target则有471 * 12 个 （第10个小时的PM2.5）

In [52]:
# 定义输入输出的大小
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype = float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            # 防止今天的数据跑到明天去
            if day == 19 and hour > 14:
                continue
            # 输入的sample大小为9*18
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1,-1)
            y[month * 471 + day + hour, 0] = month_data[month][9, day * 24 + hour + 9]


## Normalize 归一化

In [53]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis= 0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

上面的数据处理部分已懵逼！ 需要再看看咋做的 并想为啥这样操作

## Split Training Data Into "train_set" and "validation_set"
生成训练用的数据集train_set和验证数据局validation_set



In [54]:
import math
# floor() 返回数字的下舍整数。 取百分之80做训练集 剩下的 百分之20做测试集
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8):, :]
y_validation = y[math.floor(len(x) * 0.8):, :]

## Training
使用Adagrad算法作为优化器


In [55]:
# 因为常数项的存在所以dim要多加一个维度
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)

# 设置学习率
learning_rate = 100
# 迭代次数
iter_time = 1000
# adagrad 
adagrad = np.zeros([dim, 1])
eps = 0.000000001
last_loss = 0
for t in tqdm(range(iter_time)):
    # rmse
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)         
    last_loss = loss
    # 计算梯度
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
    # adgrade
    adagrad += gradient ** 2
    # 更新参数
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)

print("\n" + "Loss:" + str(last_loss))
# 保存权重
np.save('weight.npy', w)

100%|██████████| 1000/1000 [00:00<00:00, 1071.53it/s]
Loss:8.823287772292002



## Testing
导入test data 并且以训练集的方式预处理，使得测试数据形成240个维度为18\*9+1

In [57]:
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)

# 归一化
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)

## Prediction 



In [59]:
# 导入权重
w = np.load('weight.npy')
# 解决输出
ans_y = np.dot(test_x, w)

## Save Prediction to CSV File

In [62]:
import csv
with open('prediction.csv', mode='w', newline='') as prediction_file:
    csv_writer = csv.writer(prediction_file)
    header = ['id', 'value']
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)

['id', 'value']
['id_0', 2.741404287042598]
['id_1', -4.370884456673592]
['id_2', 18.26244373270194]
['id_3', 9.598128127340377]
['id_4', 2.0892488386282793]
['id_5', 1.4738494998130225]
['id_6', 7.002309849867746]
['id_7', 4.225084371439914]
['id_8', 3.1922017854384555]
['id_9', 7.0000374111473604]
['id_10', 1.6519333721442475]
['id_11', 1.3413967050490694]
['id_12', 0.8044668636103403]
['id_13', 5.335244584422716]
['id_14', 2.9786019573392304]
['id_15', -1.6292954037544511]
['id_16', 3.0581183366715585]
['id_17', 6.016887626181646]
['id_18', 0.1824775984498146]
['id_19', -0.747441981553358]
['id_20', 2.598173959640545]
['id_21', -1.183035165744311]
['id_22', 3.9779106193374396]
['id_23', 1.4861291556116853]
['id_24', 0.2242522454175493]
['id_25', 7.600797728995835]
['id_26', 8.134195848237]
['id_27', -70.80100692026932]
['id_28', 3.0767976597371054]
['id_29', 4.793725407740482]
['id_30', 12.07104554541126]
['id_31', 0.9229460126120124]
['id_32', -0.47614033491622587]
['id_33', -1.796

## 总结
结果好像不是很好哎，因为预测的pm2.5的值竟然有负数！！ 这是不可能的啊
如何优化呢， 选择不同的优化器，是不用同的model（LSTM）

In [72]:
count = 0
for i in range(len(ans_y)):
    if (ans_y[i] < 0):
        count += 1

count/len(ans_y)

0.29583333333333334